In [ ]:
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from transformers import set_seed as transformers_set_seed
from vllm import SamplingParams

In [ ]:
import ast
import copy
import gc
import gzip
import hashlib
import io
import json
import math
import os
import pickle
import random
import re
import shutil
import string
import subprocess
import sys
import textwrap
import time
import urllib
import warnings
import zipfile
from collections import Counter
from glob import glob
from typing import Any, Callable, Generator, Iterable

# import imageio
import IPython
import numpy as np
import pandas as pd
import requests
import sympy
import torch

from config import Config

In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas();

In [ ]:
cnfg = Config()

In [ ]:
class CurrentConfig:
    MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
    MAX_NEW_TOKENS = 1024
    TEMPERATURE = 0.87654321
    N_REPETITIONS = 20
    OTHER_SAMPLING_KWARGS = {"top_k": 30}

In [ ]:
curr_cnfg = CurrentConfig()

In [ ]:
def set_seed(seed: int = 42) -> None:
    """Sets the seed for generating random numbers to ensure reproducibility.

    Args:
        seed (int): The seed number. Default is 42.
    """
    transformers_set_seed(seed)

In [ ]:
set_seed()

### Utils

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## Load model

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(curr_cnfg.MODEL_PATH)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model_config = AutoConfig.from_pretrained(curr_cnfg.MODEL_PATH)
model_config.gradient_checkpointing = True

In [ ]:
model_config

LlamaConfig {
  "_name_or_path": "deepseek-ai/deepseek-math-7b-rl",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 100000,
  "eos_token_id": 100001,
  "gradient_checkpointing": true,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "moe_intermediate_size": 11008,
  "num_attention_heads": 32,
  "num_hidden_layers": 30,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.40.1",
  "use_cache": true,
  "vocab_size": 102400
}

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    curr_cnfg.MODEL_PATH,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
    config=model_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print_trainable_parameters(model)

trainable params: 839110656 || all params: 3874738176 || trainable%: 21.655931778756656


### Read data

In [ ]:
data_df = pd.read_csv(f"{cnfg.ext_comb_pre_processed}")

In [ ]:
data_df2 = pd.read_csv(f"{cnfg.base_data_path}aimo_given/train.csv")

In [ ]:
data_df

,problem_id,problem,solution,answer,link,source,level,type,stage,letter
0,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...","We can rewrite the expression as \[222,222-(22...",2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
1,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...","222,222-22,222 = 200,000\n200,000 - 2,222 = 19...",2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
2,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...","We only care about the unit's digits.\nThus, $...",2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
3,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...",We just take the units digit of each and subtr...,2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
4,085955dda8dfb374689b3f216b54d785,What is the value of this expression in decima...,We see that $\frac{44}{11}$ is $4$ $\frac{110}...,6.54,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,C
...,...,...,...,...,...,...,...,...,...,...
27856,19044_GSM8K_test,John had a son James when he was 19. James is...,Dora is 12-3=<<12-3=9>>9\nSo James is 9*2=<<9*...,8.00,NaN,GSM8K,NaN,NaN,test,NaN
27857,19045_GSM8K_test,There are some oranges in a basket. Ana spends...,There are 60 minutes in an hour. Ana peels an ...,5.00,NaN,GSM8K,NaN,NaN,test,NaN
27858,19046_GSM8K_test,Mark's car breaks down and he needs to get a n...,The discount on the radiator was 400*.8=$<<400...,230.00,NaN,GSM8K,NaN,NaN,test,NaN
27859,19047_GSM8K_test,"Farmer Brown has 20 animals on his farm, all e...",Let C be the number of chickens.\nThere are 20...,5.00,NaN,GSM8K,NaN,NaN,test,NaN


## Inference Pipeline

#### VLLM

In [ ]:
class InferenceVLLMPipeline:
    def __init__(
        self,
        model: object | None = None,
        tokenizer: object | None = None,
        stop_words: list[str] | None = None,
        model_sampling_params: dict[str, Any] | None = None,
        **kwargs,
    ) -> None:
        self.model = model
        self.tokenizer = tokenizer

        # Set stop words and fallback
        self.stop_words = stop_words
        if stop_words is None:
            self.stop_words = stop_words or [
                (
                    tokenizer.eos_token
                    if tokenizer is not None and tokenizer.eos_token is not None
                    else "</s>"
                )
            ]
        self.model_sampling_params = model_sampling_params or {}
        self.model_sampling_params["stop"] = self.stop_words

    def __call__(
        self,
        query_prompt: str | list[str],
        max_new_tokens: int,
        temperature: float,
        do_sample: bool = True,
        return_full_text: bool = False,
        stop_word_overrides: list[str] | None = None,
        n_repeats: int = 1,
        sampling_kwargs: dict[str, Any] | None = None,
        batch_style: str = "multiply",
        do_cleanup: bool = False,
    ):
        # batch style is "multiply" or "sampling_param_n"
        # Coerce into batch format
        if isinstance(query_prompt, str):
            query_prompt = [query_prompt]

        # Validate sampling is allowed and if not adjust temperature
        temperature = 0.0000001 if not do_sample else temperature

        # Get sampling parameters and update with call specific params
        sampling_params_kwargs = {**self.model_sampling_params}
        sampling_params_kwargs.update(sampling_kwargs or {})
        sampling_params_kwargs.update(
            {"temperature": temperature, "max_tokens": max_new_tokens}
        )

        # Parse input batch
        if batch_style == "multiply" and len(query_prompt) == 1:
            query_prompt = query_prompt * n_repeats
        elif batch_style == "sampling_param_n":
            sampmling_params_kwargs.update({"n": n_repeats})

        # Finalize sampling params
        _sampling_params = SamplingParams(**sampling_params_kwargs)

        # Do inference
        model_output = model.generate(query_prompt, _sampling_params)

        # Parse output
        if batch_style == "multiply":
            model_output = [output.outputs[0].text for output in model_output]
        elif batch_style == "sampling_param_n":
            model_output = [output.text for output in model_output[0].outputs]

        # Cleanup
        if do_cleanup:
            torch.cuda.empty_cache()
            gc.collect()

        return model_output

In [ ]:
def initialize_vllm_pipeline(
    model,
    tokenizer,
    stop_words: list[str] | None = None,
    model_sampling_params: dict[str, Any] | None = None,
    **kwargs,
) -> InferenceVLLMPipeline:
    """Artificial pipeline construct so we can mimic the transformers workflow.

    Args:
        model: The pre-trained model to be used for text generation.
        tokenizer: The tokenizer for text preprocessing.

    Returns:
        InferenceLLMPipeline: A configured pipeline for text generation.
    """
    return InferenceVLLMPipeline(
        model=model,
        tokenizer=tokenizer,
        stop_words=stop_words,
        model_sampling_params=model_sampling_params,
        **kwargs,
    )

In [ ]:
def run_vllm_pipeline(
    model_pipeline: InferenceVLLMPipeline,
    query_prompt: str,
    max_new_tokens: int | None = None,
    temperature: float = 0.85,
    num_repetitions: int = 2,
    sampling_kwargs: dict[str, Any] | None = None,
) -> list:
    """Executes the text-generation pipeline multiple times and collects outputs.

    Args:
        model_pipeline: The initialized text generation pipeline.
        query_prompt: Input text for the pipeline.
        max_new_tokens: Maximum number of new tokens to generate.
        temperature: Controls randomness in output generation.
        num_repetitions: Number of times to run the pipeline for each input.

    Returns:
        A list of outputs from the pipeline.
    """
    try:
        model_results = model_pipeline(
            query_prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            n_repeats=num_repetitions,
            sampling_kwargs=sampling_kwargs,
        )
    except Exception as e:
        print(f"Exception occured in `model_pipeline`: {e}")
        model_results = [
            "",
        ] * num_repetitions

    return model_results

#### Huggingface

In [ ]:
def initialize_pipeline(model, tokenizer) -> pipeline:
    """Initializes a pipeline for text generation using the provided model and tokenizer.

    Args:
        model: The pre-trained model to be used for text generation.
        tokenizer: The tokenizer for text preprocessing.

    Returns:
        pipeline: A configured pipeline for text generation.
    """
    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype="auto",
        device_map="auto",
    )

In [ ]:
def run_pipeline(
    model_pipeline: Callable,
    query_prompt: str,
    max_new_tokens: int = 2048,
    temperature: float = 0.85,
    num_repetitions: int = 2,
) -> list:
    """Executes the text-generation pipeline multiple times and collects outputs.

    Args:
        model_pipeline: The initialized text generation pipeline.
        query_prompt: Input text for the pipeline.
        max_new_tokens: Maximum number of new tokens to generate.
        temperature: Controls randomness in output generation.
        num_repetitions: Number of times to run the pipeline for each input.

    Returns:
        A list of outputs from the pipeline.
    """

    # Initialize the empty results list for this particular query prompt
    results = []

    # For N repetitions we will repeatedly attempt the problem.
    for _ in tqdm(range(num_repetitions)):
        try:
            raw_output = model_pipeline(
                query_prompt,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=temperature,
                return_full_text=False,
            )
            results.append(raw_output[0]["generated_text"])
            torch.cuda.empty_cache()
            gc.collect()
        except Exception as e:
            print(f"Exception occured in `model_pipeline`: {e}")
            results.append(None)
    return results

### Promt Engineering

In [ ]:
# def get_prediction_from_prompt(prompt):
#     input_ = tokenizer(prompt, return_tensors="pt")

#     outputs = model.generate(
#         input_ids=input_.input_ids,
#         attention_mask=input_.attention_mask,
#         max_new_tokens=150,
#     )
#     resp = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return resp

In [ ]:
# cot = """Below is a math problem you are to solve (positive numerical answer!):
# \"{}\"
# Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

In [ ]:
# problem = data_df.iloc[0].problem

In [ ]:
# problem

In [ ]:
# prompt = cot.format(problem, "{}")

In [ ]:
# answer = get_prediction_from_prompt(prompt)

In [ ]:
def prepare_problem_statement(
    problem: str,
    tool_instruction: str | None = None,
    tokenizer: Any = None,
    apply_chat_template: bool = True,
    use_simple: bool = False,
) -> str:
    """Prepares the complete problem statement by appending the tool instruction to the problem text.

    Args:
        problem (str):
            The original problem text.
        tool_instruction (str):
            Additional instructions or information to append to the problem.
        tokenizer ():
            The huggingface tokenizer
        apply_chat_template (bool, optional):
            Whether to apply the HF prompt template (requires )
            If no tokenizer is provided apply_chat_template will not work.
        use_simple (bool, optional):
            Whether to do 0 prompt engineering.



    Returns:
        A complete problem statement ready for processing.
    """
    if not use_simple and tool_instruction is not None:
        prompt_str = (
            tool_instruction
            + f"\nQUESTION:\n{problem}\n\nYou must write out the logical solution in a step by step fashion before you write any python code to solve the problem.\n\nSOLUTION:\n"
        )
    else:
        prompt_str = (
            problem
            + "\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."
        )

        #         cot = """Below is a math problem you are to solve (positive numerical answer!):
        # \"{}\"
        # Analyze this problem and think step by step to come to a solution. Do not use program. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""
    #         cot = """Below is a math problem you are to solve (positive numerical answer!):
    # \"{}\"
    # Analyze this problem and think step by step to come to a solution with program. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""
    #         prompt_str = cot.format(problem, "{}")

    if apply_chat_template:
        return tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt_str}], tokenize=False
        )
    else:
        return prompt_str

In [ ]:
def naive_parse(answer: str) -> str:
    """Extracts the last contiguous sequence of digits from a given string.

    This function is based on the function that is floating around in the top scoring code notebooks.
    I'm not sure who the original author was... but once I know I will attribute accordingly.

    Args:
        answer: A string from which to extract the digit sequence.

    Returns:
        A string containing the last sequence of digits found in the input string.
        Returns an empty string if no digits are found.

    Examples:
        naive_parse("example123test456") returns "456"
        naive_parse("no digits here!") returns ""
    """
    last_digits = ""
    found_digit = False

    for char in reversed(answer):
        if char.isdigit():
            last_digits += char
            found_digit = True
        elif found_digit:
            # Break the loop once the first non-digit is found after finding digits
            break

    # Reverse to correct the order of digits
    return last_digits[::-1]

In [ ]:
def execute_code(
    code: str,
    timeout_seconds: int = 7,
    filename: str = "code_to_execute.py",
    modulo: int = 1000,
    sympy_star_import: bool = True,
) -> int:
    """Executes the given Python code snippet and processes the output.

    Args:
        code: The Python code to execute.
        timeout_seconds: Maximum allowed time for code execution in seconds.
        filename: The filename to which the code will be written before execution.
        modulo: The modulo value to use for processing the output.
        sympy_star_import: Whether to always import everything from sympy

    Returns:
        An integer result derived from the execution output or -1 if an error occurs.
    """
    try:
        with open(filename, "w") as fout:
            fout.write(
                code if not sympy_star_import else "from sympy import *\n" + code
            )

        batcmd = f"timeout {timeout_seconds} {sys.executable} {filename}"
        shell_output = subprocess.check_output(batcmd, shell=True).decode("utf8")
        return postprocess_final_answer(shell_output, modulo)
    except Exception as e:
        print(f"Exception occured in `execute_code`: {e}")
        return -1

In [ ]:
def postprocess_final_answer(expression: str, modulo: int = 1000) -> int:
    """Postprocesses the final answer by returning the rounded/modulod value.

    Args:
        expression: The mathematical expression to evaluate as a string. (raw final answer)
        modulo: The modulo value to use in the calculation.

    Returns:
        An integer result of the evaluated expression modulo the specified value.
    """
    try:
        result = round(float(eval(expression)))
        return result % modulo
    except Exception as e:
        print(f"Exception occured in `postprocess_final_answer`: {e}")
        return -1

In [ ]:
def extract_and_process_math(result: str, modulo: int = 1000) -> int:
    """Extracts and evaluates the mathematical expression from the given string.

    Args:
        result: The string containing the expression in a LaTeX-like \\boxed{} format.
        modulo: The modulo value to use for the final result.

    Returns:
        An integer result of the evaluated expression or -1 if an error occurs.
    """
    try:
        result_output = re.findall(r"\\boxed\{(.*)\}", result)
        # print(f"Found pattern for search in box: {result_output}")
        if result_output:
            expression = result_output[-1]
        else:
            expression = naive_parse(result)

        if expression:
            return postprocess_final_answer(expression, modulo)
        return -1
    except Exception as e:
        print(f"Exception occured in `extract_and_process_math`: {e}")
        return -1

In [ ]:
def process_output(
    output: str,
    timeout_seconds: int = 7,
    filename: str = "code_to_execute.py",
    modulo: int = 1000,
) -> tuple:
    """Processes the provided output string to execute contained code and extract mathematical results.

    Args:
        output: The string that may contain Python code in triple backticks and/or a mathematical expression in \\boxed{}.
        timeout_seconds: Maximum allowed time for code execution in seconds.
        filename: The filename for saving and executing the Python code.
        modulo: The modulo value to use for processing the outputs.

    Returns:
        A tuple (result_output, code_output) where each is an integer result of the processing or -1 if an error occurs.
    """
    code_output = -1
    result_output = -1

    # Extract and execute code from output
    code_pattern = re.compile(r"```(?:\S*?\n)?(.*?)```", re.DOTALL)
    code_match = code_pattern.search(output)
    # print(f"Code Match: {code_match}")
    if code_match:
        code = code_match.group(1)
        # print(f"\n=========== Found Code:{code} ===========")
        code_output = execute_code(code, timeout_seconds, filename, modulo)
        # print("CODE RESULTS", code_output)

    # Extract and process mathematical result
    result_output = extract_and_process_math(output, modulo)
    # print("BOXED", result_output)

    return result_output, code_output

In [ ]:
def aggregate_results(
    code_results: Iterable,
    boxed_results: Iterable,
    boxed_copies_over_code_fail: bool = True,
    use_code_and_boxed: bool = False,
) -> int:
    """Aggregates the outputs, resolves errors, and determines the most common valid output.

    Args:
        code_results: List of code outputs.
        boxed_results: List of mathematical result outputs.
        boxed_copies_over_code_fail: Whether the non-error boxed results will copy over the failed code results
        use_code_and_boxed: Whether to aggregate results from both code and boxed results initially

    Returns:
        The most common valid output among the provided answers or -1 if none are valid.
    """
    # So we can pop
    code_results, boxed_results = list(code_results), list(boxed_results)
    if all(x < 0 for x in code_results):
        boxed_copies_over_code_fail = True

    # Get the results array
    if boxed_copies_over_code_fail and not use_code_and_boxed:
        results = []
        for i in range(len(code_results) - 1, -1, -1):
            if code_results[i] > 0:
                results.append(code_results[i])
            else:
                code_results.pop(i)
                results.append(boxed_results.pop(i))
        results = results[::-1]
    elif not boxed_copies_over_code_fail and not use_code_and_boxed:
        results = code_results
    else:
        results = code_results + boxed_results
    results = np.array(results)

    # Handle negatives as invalid results and handle negatives in boxed_results if needed
    results = np.where(results < 0, -1, results)
    boxed_results = np.where(np.array(boxed_results) < 0, -1, np.array(boxed_results))

    # Get most common
    most_common_results_w_counts = [
        x for x in Counter(results).most_common() if x[0] != -1
    ]
    if len(most_common_results_w_counts) == 0:
        return 1
    elif len(most_common_results_w_counts) == 1:
        return int(abs(most_common_results_w_counts[0][0]))
    if (
        most_common_results_w_counts[0][1] == most_common_results_w_counts[1][1]
        and not use_code_and_boxed
    ):
        most_common_results_w_counts = [
            x
            for x in Counter(
                np.concatenate(
                    (
                        results,
                        results,
                        results,
                        results,
                        results,
                        results,
                        results,
                        boxed_results,
                    )
                )
            ).most_common()
            if x[0] != -1
        ]
    return int(abs(most_common_results_w_counts[0][0]))

In [ ]:
# text_gen_pipeline = initialize_vllm_pipeline(
#     model, tokenizer, model_sampling_params=curr_cnfg.OTHER_SAMPLING_KWARGS
# )

text_gen_pipeline = initialize_pipeline(model, tokenizer)

In [ ]:
# tmp = data_df.iloc[101]
# problem = tmp.problem
# solution = tmp.solution
# print(problem)
# print(solution)

In [ ]:
tmp = data_df2.iloc[7]
problem = tmp.problem
solution = tmp.answer
print(problem)
print(solution)

The points $\left(x, y\right)$ satisfying $((\vert x + y \vert - 10)^2 + ( \vert x - y \vert - 10)^2)((\vert x \vert - 8)^2 + ( \vert y \vert - 8)^2) = 0$ enclose a convex polygon. What is the area of this convex polygon?
320


In [ ]:
TOOL_INSTRUCTIONS = None

In [ ]:
# prompt = prepare_problem_statement(
#     problem,
#     tool_instruction=TOOL_INSTRUCTIONS,
#     tokenizer=None,
#     apply_chat_template=None,
# )
# print(prompt)

In [ ]:
prompt = prepare_problem_statement(
    problem,
    tool_instruction=TOOL_INSTRUCTIONS,
    tokenizer=tokenizer,
    apply_chat_template=True,
)
print(prompt)

<｜begin▁of▁sentence｜>User: Below is a math problem you are to solve (positive numerical answer!):
"The points $\left(x, y\right)$ satisfying $((\vert x + y \vert - 10)^2 + ( \vert x - y \vert - 10)^2)((\vert x \vert - 8)^2 + ( \vert y \vert - 8)^2) = 0$ enclose a convex polygon. What is the area of this convex polygon?"
Analyze this problem and think step by step to come to a solution. Do not use program. After solving the problem, output the final numerical answer within \boxed{}.






In [ ]:
# # answer = get_prediction_from_prompt(prompt)
# results = run_vllm_pipeline(
#     model_pipeline=text_gen_pipeline,
#     query_prompt=prompt,
#     num_repetitions=curr_cnfg.N_REPETITIONS,
#     temperature=curr_cnfg.TEMPERATURE,
#     max_new_tokens=curr_cnfg.MAX_NEW_TOKENS,
# )

In [ ]:
results = run_pipeline(
    model_pipeline=text_gen_pipeline,
    query_prompt=prompt,
    num_repetitions=curr_cnfg.N_REPETITIONS,
    temperature=curr_cnfg.TEMPERATURE,
    max_new_tokens=curr_cnfg.MAX_NEW_TOKENS,
)

  0%|          | 0/7 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [ ]:
# results

In [ ]:
processed_demo_results = [
    process_output(demo_result) for demo_result in results if demo_result
]
demo_boxed_results, demo_code_results = (
    zip(*processed_demo_results) if processed_demo_results else ([], [])
)
demo_final_answer = aggregate_results(
    demo_code_results, demo_boxed_results, boxed_copies_over_code_fail=True
)

In [ ]:
print(f"\n... RAW CODE RESULTS        : {demo_code_results}")
print(f"... RAW BOXED RESULTS       : {demo_boxed_results}")
print(f"... AGGREGATED FINAL RESULT : {demo_final_answer}\n\n")


... RAW CODE RESULTS        : (-1, -1, -1, -1, -1, -1, -1)
... RAW BOXED RESULTS       : (200, 320, 164, 100, 256, 100, 400)
... AGGREGATED FINAL RESULT : 100




In [ ]:
# answer

In [ ]:
# process_output(answer)

### Val Pipeline

In [ ]:
def run_on_df(df):
    df = df.copy()
    for i, row in df.iterrows():
        print(f"\n =========== Problem {i+1} ============")
        problem = row.problem
        solution = row.answer
        print(f"\n Problem: {problem} ")
        print(f"\nAnswer: {solution}")

        prompt = prepare_problem_statement(
            problem,
            tool_instruction=TOOL_INSTRUCTIONS,
            tokenizer=tokenizer,
            apply_chat_template=True,
        )
        print(f"\n Prompt: {prompt} ")

        results = run_pipeline(
            model_pipeline=text_gen_pipeline,
            query_prompt=prompt,
            num_repetitions=curr_cnfg.N_REPETITIONS,
            temperature=curr_cnfg.TEMPERATURE,
            max_new_tokens=curr_cnfg.MAX_NEW_TOKENS,
        )

        processed_demo_results = [
            process_output(demo_result) for demo_result in results if demo_result
        ]
        demo_boxed_results, demo_code_results = (
            zip(*processed_demo_results) if processed_demo_results else ([], [])
        )
        demo_final_answer = aggregate_results(
            demo_code_results, demo_boxed_results, boxed_copies_over_code_fail=True
        )
        print(f"\n... RAW CODE RESULTS        : {demo_code_results}")
        print(f"... RAW BOXED RESULTS       : {demo_boxed_results}")
        print(f"... AGGREGATED FINAL RESULT : {demo_final_answer}\n\n")
        df["pred"] = demo_final_answer

    return df

In [ ]:
reslt_df = run_on_df(data_df2)


 =========== Problem 1 ============

 Problem: Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin? 

Answer: 52

 Prompt: <｜begin▁of▁sentence｜>User: Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \boxed{}.

 


  0%|          | 0/20 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for

Exception occured in `postprocess_final_answer`: name 'sqrt' is not defined
Exception occured in `postprocess_final_answer`: name 'k' is not defined


Traceback (most recent call last):
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 32, in <module>
    result = sum_of_squares()
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 19, in sum_of_squares
    solutions = solveset(equation, x, domain=S.Reals)
UnboundLocalError: local variable 'S' referenced before assignment
Traceback (most recent call last):
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 23, in <module>
    result = sum_of_squares_of_distances()
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 9, in sum_of_squares_of_distances
    solutions = solve(parabola_eq, x)
UnboundLocalError: local variable 'solve' referenced before assignment


...

  0%|          | 0/20 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for

Exception occured in `execute_code`: Command 'timeout 7 /home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/bin/python3 code_to_execute.py' returned non-zero exit status 1.


Traceback (most recent call last):
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 25, in <module>
    result = find_f_of_100()
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 16, in find_f_of_100
    if f(2*i)!= 2*f(i) + 1:
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 18, in <lambda>
    f = lambda n: n if n!= i else (2*f(i) + 1)
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 18, in <lambda>
    f = lambda n: n if n!= i else (2*f(i) + 1)
  File "/home/rohit/Desktop/rohit/kaggle_llm/aimo/code_to_execute.py", line 18, in <lambda>
    f = lambda n: n if n!= i else (2*f(i) + 1)
  [Previous line repeated 995 more times]
RecursionError: maximum recursion depth exceeded in comparison


Exception occured in `execute_code`: Command 'timeout 7 /home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/bin/python3 code_to_execute.py' returned non-zero exit status 1.
Exception occured in `postprocess_final_answer`: invalid syntax (<string>, line 1)

... RAW CODE RESULTS        : (-1, 393, 793, 775, 201, 793, 999, 127, 775, -1, -1, -1, 793, 999, -1, 775, 201, -1, -1, -1)
... RAW BOXED RESULTS       : (767, 393, 793, 617, 201, 793, 7, 199, 193, 8, 199, 159, 793, 793, 87, 7, 201, 1, 983, 3)
... AGGREGATED FINAL RESULT : 793


